In [27]:
import openpyxl
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time

options = Options()
options.headless = True
b = webdriver.Chrome(options=options, executable_path="chromium.chromedriver")

from bs4 import BeautifulSoup

In [2]:
df = pd.read_excel("SampleMadisonRegistrations.xlsx")
df[:5]

,Entity Type,EntityID,EntityName,Inc/OrgDate,EntityStatus,StatusDate,RegAgentofficeCity
0,2,N051411,"#1 A LIFESAFER DISTRIBUTION, INC.",2019-11-13,INC,2019-11-13,MADISON
1,1,N024571,"#1 A LIFESAFER OF WISCONSIN, INC.",1995-11-28,IGS,2019-10-14,MADISON
2,12,N042402,#BEMADISON LLC,2013-10-16,ADS,2018-04-24,MADISON
3,12,N047607,#DOBETTER LLC,2017-08-23,DLQ,2019-07-01,MADISON
4,6,E047252,"(""EVERY WOMAN NEEDS A MARGARET"") - EWNAM, INC.",2015-06-04,ADS,2019-06-10,MADISON


In [3]:
#df["EntityName"].head()

In [4]:
#Tyler's suggestion: search for madison street yardwaste to find keywords signifying street names
#split the strings, find non-numbers and words (delete them -- replace them with space) 
#then ask does it contain a number or the keyword in the address list

## Extracting street signifiers from https://www.cityofmadison.com/streets/yardWaste/leaf/

In [5]:
url = "https://www.cityofmadison.com/streets/yardWaste/leaf/"
b.get(url)
b.find_element_by_id("StType") #returns a WebElement object

<selenium.webdriver.remote.webelement.WebElement (session="835b4652b25ce766ba2cda17cfdce780", element="a4dd987a-95e5-4a37-ae3d-33c24f563b7a")>

In [6]:
#get the html returned by Selenium
print(b.page_source[:100]) 

<html lang="en"><!-- InstanceBegin template="/Templates/streets960.dwt" codeOutsideHTMLIsLocked="fal


In [7]:
#extract select element from the html using BeautifulSoup
page = BeautifulSoup(b.page_source, "html.parser")
street_name_options = page.find_all("select")
assert len(street_name_options) == 1
type(street_name_options[0]) #street_name_options is now a list with one bs4 element

bs4.element.Tag

In [8]:
street_name_options

[<select id="StType" name="StType">
 <option value=" ">
 </option><option value="Aly">Aly 
 			</option><option value="Ave">Ave 
 			</option><option value="Blvd">Blvd 
 			</option><option value="Bnd">Bnd 
 			</option><option value="Cir">Cir 
 			</option><option value="Cres">Cres 
 			</option><option value="Ct">Ct 
 			</option><option value="Dr">Dr 
 			</option><option value="Gln">Gln 
 			</option><option value="Grn">Grn 
 			</option><option value="Hts">Hts 
 			</option><option value="Hwy">Hwy 
 			</option><option value="Ln">Ln 
 			</option><option value="Loop">Loop 
 			</option><option value="Mall">Mall 
 			</option><option value="Pass">Pass 
 			</option><option value="Path">Path 
 			</option><option value="Pkwy">Pkwy 
 			</option><option value="Pl">Pl 
 			</option><option value="Plz">Plz 
 			</option><option value="Ramp">Ramp 
 			</option><option value="Rd">Rd 
 			</option><option value="Rdg">Rdg 
 			</option><option value="Row">Row 
 			</option><option value="R

In [21]:
street_signifiers = []
for line in street_name_options[0]:
    street_signifiers.append(line.get_text().strip())
#print(len(street_signifiers))
street_signifiers

street_signifiers_filtered = list(filter(None, street_signifiers)) #a list of names without None values

In [20]:
street_signifiers_cap = []
for item in street_signifiers_filtered:
    street_signifiers_cap.append(item.upper())
street_signifiers_cap

['ALY',
 'AVE',
 'BLVD',
 'BND',
 'CIR',
 'CRES',
 'CT',
 'DR',
 'GLN',
 'GRN',
 'HTS',
 'HWY',
 'LN',
 'LOOP',
 'MALL',
 'PASS',
 'PATH',
 'PKWY',
 'PL',
 'PLZ',
 'RAMP',
 'RD',
 'RDG',
 'ROW',
 'RR',
 'RUN',
 'SPUR',
 'SQ',
 'ST',
 'TER',
 'TRCE',
 'TRL',
 'VW',
 'WALK',
 'WAY',
 'XING']

## Matching street signifiers to addresses in SampleMadisonRegistrations.xlsx

In [11]:
df = pd.read_excel("SampleMadisonRegistrations.xlsx")
df[:5]

,Entity Type,EntityID,EntityName,Inc/OrgDate,EntityStatus,StatusDate,RegAgentofficeCity
0,2,N051411,"#1 A LIFESAFER DISTRIBUTION, INC.",2019-11-13,INC,2019-11-13,MADISON
1,1,N024571,"#1 A LIFESAFER OF WISCONSIN, INC.",1995-11-28,IGS,2019-10-14,MADISON
2,12,N042402,#BEMADISON LLC,2013-10-16,ADS,2018-04-24,MADISON
3,12,N047607,#DOBETTER LLC,2017-08-23,DLQ,2019-07-01,MADISON
4,6,E047252,"(""EVERY WOMAN NEEDS A MARGARET"") - EWNAM, INC.",2015-06-04,ADS,2019-06-10,MADISON


In [28]:
num_of_LLCs = len(df["EntityName"])
num_of_LLCs

2911

In [31]:
#I want to check if any of the street_signifiers_cap is in EntityName
for signifier in street_signifiers_cap: 
    if len(signifier) > 2: 
        for name in df["EntityName"][10:50]:
            if signifier in name:   
                print(name)   

100 WISCONSIN AVENUE CONDOMINIUM OWNERS ASSOCIATION, INC.
1000 W. NORTHLAND AVE LLC
1008 WAUGOO AVENUE, LLC
1016 COLAN BLVD, LLC
10 - 12 BAHR CIRCLE LLC
102 PRAIRIE HEIGHTS LLC
1010 WISCONSIN DELLS PARKWAY SOUTH LAKE DELTON, WI LLC


In [32]:
for signifier in street_signifiers_cap: 
    for name in df["EntityName"][10:50]:
        if signifier in name:   #need to modify this so that PL, ST acronyms are not counted
            print(name) 

100 WISCONSIN AVENUE CONDOMINIUM OWNERS ASSOCIATION, INC.
1000 W. NORTHLAND AVE LLC
1008 WAUGOO AVENUE, LLC
1016 COLAN BLVD, LLC
10 - 12 BAHR CIRCLE LLC
1014 S. LAKE SHORE DRIVE LLC
1040 QUINN DRIVE, LLC
102 PRAIRIE HEIGHTS LLC
10 NEWTONS WOMEN'S WELLNESS FOUNDATION, INC.
100% PLATINUM CONSULTING LLC
100 SOUTH FIRST, LLC
1000 EAST STREET, LLC
100STATE INC.
101 OSTHOFF, LLC
1010 ANN STREET LLC
1011 NORTH WISCONSIN STREET OWNER LLC
1021-23 N. 24TH ST. LLC
1024 MAIN STREET, LLC
1028 ERIN ST LLC
103 LIBERTY STREET LLC
103 S RIDGE STREET LLC
105 N MAIN STREET LLC
1010 WISCONSIN DELLS PARKWAY SOUTH LAKE DELTON, WI LLC


## See which LLCs match to LLCs that opened an eviction case in June 2021 (ccapoutputfile_)

In [16]:
#split the strings, find non-numbers and words (delete them -- replace them with space) 
#then ask does it contain a number or the keyword in the address list

In [17]:
#Identify key words that signify address
address_signifiers = ["NORTH", "SOUTH", "EAST", "WEST", "N", "N.", "S", "S.", "E", "E.", "W", "W.", "STREET", "ST", "AVE", "DRIVE", "RD", "PLACE", "PL", "LANE", "COURT", "CT", "WAY", "HIGHWAY"]

In [18]:
#question: 
#for addresses like 1236 Willy LLC, it could correspond to an address even though it doesn't contain the street keywords
#or like 134 Fair Oaks, LLC (it sounds like a street address)